In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os 
os.chdir("/content/drive/MyDrive/Master's/Master's SEM 1/Machine learning/BMED6517-Fall22-Team7/Datasets/Head CT/")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf 
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from PIL import Image

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Master's/Master's SEM 1/Machine learning/BMED6517-Fall22-Team7/Datasets/Head CT/labels.csv")

In [ ]:
df.head()

,id,hemorrhage
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


In [ ]:
files=glob.glob("/content/drive/MyDrive/Master's/Master's SEM 1/Machine learning/BMED6517-Fall22-Team7/Datasets/Head CT/head_ct/head_ct/*.png")
SIZE = 256

In [ ]:
label=[]
img_array=[]
for i in files:
  sub_heading=i.split('/')
  sub_sub_heading=sub_heading[12].split('.')
  index=int(sub_sub_heading[0])
  img=cv2.imread(i,0)
  img= Image.fromarray(img)
  img = img.resize((SIZE, SIZE))
  img_array.append(np.array(img))
  label.append(df[' hemorrhage'][index])

In [ ]:
x=np.array(img_array)
x=x/255
y=np.array(label)

In [ ]:
x=np.expand_dims(x,3)
x.shape

(200, 256, 256, 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.15, random_state = 0)

In [ ]:
class Block(tf.keras.Model):
    def __init__(self, filters, kernel_size, repetitions, pool_size=2, strides=2):
        super(Block, self).__init__()
        self.filters =filters
        self.kernel_size = kernel_size
        self.repetitions = repetitions
        
        # Define a conv2D_0, conv2D_1, etc based on the number of repetitions
        for i in range(self.repetitions):
            
            # Define a Conv2D layer, specifying filters, kernel_size, activation and padding.
            vars(self)[f'conv2D_{i}'] = tf.keras.layers.Conv2D(self.filters,self.kernel_size,activation='relu',padding="same")
        
        # Define the max pool layer that will be added after the Conv2D blocks
        self.max_pool = tf.keras.layers.MaxPooling2D(pool_size, strides=strides)
  
    def call(self, inputs):
        # access the class's conv2D_0 layer
        conv2D_0 = vars(self)['conv2D_0']
        
        # Connect the conv2D_0 layer to inputs
        x = conv2D_0(inputs)

        # for the remaining conv2D_i layers from 1 to `repetitions` they will be connected to the previous layer
        for i in range(1,self.repetitions):
            # access conv2D_i by formatting the integer `i`. (hint: check how these were saved using `vars()` earlier)
            conv2D_i = vars(self)[f'conv2D_{i}']
            
            # Use the conv2D_i and connect it to the previous layer
            x = conv2D_i(x)

        # Finally, add the max_pool layer
        max_pool = self.max_pool(x)
        
        return max_pool

In [ ]:
class MyVGG(tf.keras.Model):

    def __init__(self, num_classes):
        super(MyVGG, self).__init__()

        # Creating blocks of VGG with the following 
        # (filters, kernel_size, repetitions) configurations
        self.block_a = Block(filters=64, kernel_size=3, repetitions=2)
        self.block_b = Block(filters=128, kernel_size=3, repetitions=2)
        self.block_c = Block(filters=256, kernel_size=3, repetitions=3)
        self.block_d = Block(filters=512, kernel_size=3, repetitions=3)
        self.block_e = Block(filters=512, kernel_size=3, repetitions=3)

        # Classification head
        # Define a Flatten layer
        self.flatten = tf.keras.layers.Flatten()
        # Create a Dense layer with 256 units and ReLU as the activation function
        self.fc = tf.keras.layers.Dense(256,activation='relu')
        # Finally add the softmax classifier using a Dense layer
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        # Chain all the layers one after the other
        x = self.block_a(inputs)
        x = self.block_b(x)
        x = self.block_c(x)
        x = self.block_d(x)
        x = self.block_e(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.classifier(x)
        return x

In [ ]:
vgg = MyVGG(num_classes=2)
vgg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
vgg.fit(X_train,y_train,epochs=5)

Epoch 1/5
6/6 [==============================] - 122s 19s/step - loss: 0.6881 - accuracy: 0.5059
Epoch 2/5
6/6 [==============================] - 118s 19s/step - loss: 0.6899 - accuracy: 0.5059
Epoch 3/5
6/6 [==============================] - 121s 19s/step - loss: 0.6721 - accuracy: 0.6294
Epoch 4/5
6/6 [==============================] - 133s 22s/step - loss: 0.6543 - accuracy: 0.7176
Epoch 5/5
6/6 [==============================] - 117s 19s/step - loss: 0.6466 - accuracy: 0.6059


In [ ]:
vgg.save_weights("/content/drive/MyDrive/Master's/Master's SEM 1/Machine learning/BMED6517-Fall22-Team7/Models/vgg.h5")

In [ ]:
y_pred=vgg.predict(X_test)

1/1 [==============================] - 20s 20s/step


In [ ]:
y_pred_binary=[]
for a in y_pred:
  y_pred_binary.append(np.argmax(a))

y_pred_binary=np.array(y_pred_binary)

In [ ]:
from sklearn import metrics
print("Accuracy of model=",metrics.accuracy_score(y_test,y_pred_binary))

Accuracy of model= 0.5666666666666667


In [ ]:
from sklearn.metrics import classification_report
target_names = ['0','1']
print(classification_report(y_test, y_pred_binary, target_names=target_names))

              precision    recall  f1-score   support

           0       1.00      0.19      0.32        16
           1       0.52      1.00      0.68        14

    accuracy                           0.57        30
   macro avg       0.76      0.59      0.50        30
weighted avg       0.78      0.57      0.49        30



In [ ]:
pip install tf-keras-vis tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install keras-vis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
y_pred_list=y_pred_binary.tolist()

In [ ]:
X_test_copy=X_test.copy()

In [ ]:
from tf_keras_vis.utils.scores import CategoricalScore
score = CategoricalScore(y_pred_list)

In [ ]:
from matplotlib import cm
from tf_keras_vis.gradcam import Gradcam
from tensorflow.keras.applications.vgg16 import preprocess_input


input_classes = ['0','1']''

input_images = preprocess_input(X_test_copy)

# Create Gradcam object
gradcam = Gradcam(vgg,
                  clone=True)

# Generate heatmap with GradCAM
cam = gradcam(score,
              input_images,
              penultimate_layer=-1)

IndexError: ignored